In [1]:
import os
import pandas as pd
from data_ingestion import setup_knowledge_base  # step1_loaddata
from chunking import text_splitter  # step2_chunking
from raptor import RaptorPipeline  # step3_RAPTOR
from utils import convert_df_to_documents
from model_config import VietnameseEmbeddings
from langchain_community.document_loaders import TextLoader
from typing import List
from tqdm import tqdm
import glob
class TXTProcessor:
    def __init__(self, directory: str = None, encoding: str = 'utf-8'):
        self.directory = directory
        self.encoding = encoding
        self.file_paths = []

    def get_txt_files(self) -> list:
        if not self.directory:
            raise ValueError("Directory is not specified.")
        self.file_paths = glob.glob(os.path.join(self.directory, "*.txt"))
        return self.file_paths

    def setup_txt(self, file_path: str) -> list:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        file_name = os.path.basename(file_path)
        loader = TextLoader(file_path, encoding=self.encoding)
        loaded_docs = loader.load()
        for doc in loaded_docs:
            doc.metadata['source'] = file_name
        return loaded_docs

# Các bước pipeline
def step1_load_data(txt_file: str) -> List:
    """
    Load dữ liệu từ một file .txt cụ thể.
    """
    processor = TXTProcessor()
    documents = processor.setup_txt(txt_file)
    print(f"Đã load dữ liệu từ file: {txt_file}")
    return documents

def step2_chunking(documents):
    """
    Chunk tài liệu thành các phần nhỏ hơn.
    """
    chunks = text_splitter.split_documents(documents)
    for i, chunk in enumerate(chunks):
        chunk.metadata["id"] = str(i)
    chunks_metadata = [chunk.metadata for chunk in chunks]
    chunks_content = [chunk.page_content for chunk in chunks]
    return chunks_metadata, chunks_content

def step3_RAPTOR(chunks_metadata, chunks_content):
    """
    Thực hiện RAPTOR pipeline trên chunks.
    """
    raptor = RaptorPipeline()
    results = raptor.recursive_embed_cluster_summarize(chunks_content, chunks_metadata, level=1, n_levels=3)
    final_df = raptor.build_final_dataframe(results)
    return final_df

def process_directory(directory: str):
    """
    Đọc và xử lý toàn bộ các file trong thư mục với RAPTOR.
    """
    processor = TXTProcessor(directory=directory)
    txt_files = processor.get_txt_files()
    
    for txt_file in tqdm(txt_files, desc="Processing files"):
        # Bước 1: Load dữ liệu
        documents = step1_load_data(txt_file)
        
        # Bước 2: Chunking
        chunks_metadata, chunks_content = step2_chunking(documents)
        
        # Bước 3: RAPTOR
        final_df = step3_RAPTOR(chunks_metadata, chunks_content)
        
        # Lưu kết quả thành file CSV
        output_csv = f"{os.path.splitext(txt_file)[0]}_results.csv"
        final_df.to_csv(output_csv, index=False)
        print(f"Đã lưu kết quả RAPTOR cho file {txt_file} vào {output_csv}")
        # # Lưu kết quả thành file Pickle
        # output_pkl = f"{os.path.splitext(txt_file)[0]}_results.pkl"
        # final_df.to_pickle(output_pkl)  # Lưu DataFrame dưới dạng Pickle
        # print(f"Đã lưu kết quả RAPTOR cho file {txt_file} vào {output_pkl}")
# Thực thi pipeline
if __name__ == "__main__":
    directory_path = r"langchain_community"  # Thay bằng đường dẫn thực tế
    process_directory(directory_path)


c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Processing files:   0%|          | 0/6 [00:00<?, ?it/s]

Đã load dữ liệu từ file: D:\DATN\QA_System\data_hust\test\Hướng dẫn làm thủ tục thanh toán ra Trường.txt
Initializing Vietnamese embedding model: dangvantuan/vietnamese-embedding


Processing files:  17%|█▋        | 1/6 [00:18<01:30, 18.02s/it]

Đã lưu kết quả RAPTOR cho file D:\DATN\QA_System\data_hust\test\Hướng dẫn làm thủ tục thanh toán ra Trường.txt vào D:\DATN\QA_System\data_hust\test\Hướng dẫn làm thủ tục thanh toán ra Trường_results.csv
Đã load dữ liệu từ file: D:\DATN\QA_System\data_hust\test\Hướng dẫn sử dụng phần mềm Office 365 và học trực tuyến; sử dụng hệ thống email do Trường cấp.txt


Processing files:  33%|███▎      | 2/6 [00:33<01:05, 16.28s/it]

...

Đã lưu kết quả RAPTOR cho file D:\DATN\QA_System\data_hust\test\Hướng dẫn sử dụng Thư viện Tạ Quang Bửu.txt vào D:\DATN\QA_System\data_hust\test\Hướng dẫn sử dụng Thư viện Tạ Quang Bửu_results.csv
Đã load dữ liệu từ file: D:\DATN\QA_System\data_hust\test\Hướng dẫn thủ tục, biểu mẫu, thắc mắc về học tập, học phí.txt


Processing files:  67%|██████▋   | 4/6 [01:03<00:31, 15.54s/it]

Đã lưu kết quả RAPTOR cho file D:\DATN\QA_System\data_hust\test\Hướng dẫn thủ tục, biểu mẫu, thắc mắc về học tập, học phí.txt vào D:\DATN\QA_System\data_hust\test\Hướng dẫn thủ tục, biểu mẫu, thắc mắc về học tập, học phí_results.csv
Đã load dữ liệu từ file: D:\DATN\QA_System\data_hust\test\Hướng dẫn tổ chức đánh giá kết quả rèn luyện.txt


Processing files:  67%|██████▋   | 4/6 [02:05<01:02, 31.28s/it]


IndexError: index out of range in self

In [2]:
import pandas as pd
import glob
import os

def merge_csv_files(input_directory: str, output_file: str):
    # Tạo danh sách tất cả các tệp CSV trong thư mục
    csv_files = glob.glob(os.path.join(input_directory, "*.csv"))
    
    # Đọc và ghép tất cả các tệp CSV lại thành một DataFrame
    df_list = []  # Danh sách để lưu các DataFrame
    for file in csv_files:
        df = pd.read_csv(file)  # Đọc tệp CSV
        df_list.append(df)  # Thêm DataFrame vào danh sách
    
    # Ghép tất cả các DataFrame lại thành một
    merged_df = pd.concat(df_list, ignore_index=True)  # ignore_index để không giữ chỉ số ban đầu
    print(f"Tất cả các tệp CSV đã được ghép vào {output_file}")
    # Lưu DataFrame đã ghép vào tệp CSV mới
    merged_df.to_csv(output_file, index=False) 
    df = merged_df
    df = df.drop(columns=['embedding'])
    df.head()
    # Lưu lại DataFrame vào excel mới
    df.to_excel(execfile, index=False)
    

# Sử dụng hàm
input_directory = r"D:\DATN\QA_System\data_hust\test"  # Thư mục chứa các tệp CSV
output_file = r"D:\DATN\QA_System\data_hust\test\merged_output.csv"  # Tên tệp CSV đầu ra
execfile = r"D:\DATN\QA_System\data_hust\test\finaldf.xlsx"
merge_csv_files(input_directory, output_file)


Tất cả các tệp CSV đã được ghép vào D:\DATN\QA_System\data_hust\test\merged_output.csv
